In [ ]:
%pip install clip-retrieval img2dataset pandas numpy 

In [2]:
%pip install torchvision>=0.10.1,<2 --index-url https://download.pytorch.org/whl/cu117

Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.


In [31]:
import torch
import os
import pandas as pd
import numpy as np
from clip_retrieval import clip_inference
import shutil
import torch
from torch.utils.data import TensorDataset, DataLoader

base_folder = os.path.join("F:", "nude_sexy_safe_v1_x320", "testing")
output_base_folder = os.path.join("F:", "nsfw_embeddings")
categories = ["nude", "sexy", "safe"]

True


In [ ]:
for i, c in enumerate(categories):
    folder = os.path.join(base_folder, c)
    output_folder = os.path.join(output_base_folder, c)
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    # https://github.com/rom1504/clip-retrieval/issues/220
    clip_inference(folder, output_folder, num_prepro_workers=0, clip_model="ViT-L/14", batch_size=32)

In [72]:
cat_embeddings = {}
for c in categories:
    cat_embeddings[c] = np.load(os.path.join(output_base_folder, c, "img_emb", "img_emb_0.npy"))


def npy_to_tensor_dataset(pos_x, neg_x):

    max_len = min(pos_x.shape[0], neg_x.shape[0])
    pos_x, neg_x = pos_x[:max_len], neg_x[:max_len]

    num_pos = pos_x.shape[0]
    num_neg = neg_x.shape[0]

    pos_y = np.ones((num_pos, 1))
    neg_y = np.zeros((num_neg, 1))

    x_np = np.vstack((pos_x, neg_x))
    y_np = np.vstack((pos_y, neg_y))

    return torch.from_numpy(x_np).type(torch.float16), torch.from_numpy(y_np).type(torch.float16)

In [93]:
# pos_x = torch.from_numpy(np.load(r"F:\nsfw_testset\drawings-test\img_emb\img_emb_0.npy"))
# neg_x = torch.from_numpy(np.load(r"F:\nsfw_testset\neutral-test\img_emb\img_emb_0.npy"))

pos_x_val = cat_embeddings["nude"]
neg_x_val = cat_embeddings["safe"]

x, y = npy_to_tensor_dataset(pos_x_val, neg_x_val)

val_dataset = TensorDataset(x, y)

# Define the data loaders
train_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam, AdamW, Adagrad
from tqdm import tqdm
import wandb

class H14_NSFW_Detector(nn.Module):
    def __init__(self, input_size=1024):
        super().__init__()
        self.input_size = input_size
        self.layers = nn.Sequential(
            nn.Linear(self.input_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 16),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        return self.layers(x)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = H14_NSFW_Detector(768).to(device)
model.train()
criterion = nn.MSELoss().to(device)
optimizer = Adam(model.parameters(), lr=3e-5)
num_epochs = 10

epoch = 0

# wandb.init(project="nsfw-detefctor")
# wandb.watch(model)

with torch.cuda.amp.autocast():
    # Training loop
    for epoch in range(num_epochs):
        train_iter = tqdm(train_loader, total=len(train_loader))
        for inputs, labels in train_iter:
            inputs = inputs.to(device)
            labels = labels.to(device)

            inputs_list, labels_list = inputs.tolist(), labels.tolist()
            optimizer.zero_grad()

            outputs = model(inputs)
            outputs_list = outputs.tolist()

            loss = criterion(outputs, labels)
            log_dict = {"loss": loss.item(), "epoch": epoch, "lr": optimizer.param_groups[0]['lr']}
            train_iter.set_postfix(log=log_dict)
            # wandb.log(log_dict)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

In [116]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
import numpy as np
from joblib import dump, load

pos_x_val = np.concatenate([cat_embeddings["nude"], cat_embeddings["sexy"]])
neg_x_val = cat_embeddings["safe"]

clf = SVC()
# clf = AdaBoostClassifier()
clf.fit(x, y)
x, y = npy_to_tensor_dataset(pos_x_val, neg_x_val)

pos_val_categories = ["porn-test", "sexy-test", "drawings-test", "hentai-test", "neutral-test"]
pos_x_val = []
for c in pos_val_categories:
    pos_x_val.append(torch.from_numpy(np.load(os.path.join(r"F:", "nsfw_testset", c, "img_emb", "img_emb_0.npy"))))
pos_x_val = torch.cat(pos_x_val)
neg_x_val = torch.from_numpy(np.load(r"F:\nsfw_testset\neutral-test\img_emb\img_emb_0.npy"))

x_val, y_val = npy_to_tensor_dataset(pos_x_val, neg_x_val)
y_val_list = y_val.squeeze().tolist()

vals = clf.predict(x_val)
print(y_val_list)
print(list(vals))
acc = np.sum(vals == y_val_list) / len(vals)
print(acc)

dump(clf, r"F:\nsfw_testset\nsfw_classifier.joblib")

c:\Users\coold\Documents\GitHub\clip-nsfw-detector\.venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [124]:
import clip
import urllib
import io
from PIL import Image as pimage

model, preprocess = clip.load("ViT-L/14", device="cpu", jit=True)

def download_image(url):
    urllib_request = urllib.request.Request(
        url,
        data=None,
        headers={"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:72.0) Gecko/20100101 Firefox/72.0"},
    )
    with urllib.request.urlopen(urllib_request, timeout=10) as r:
        img_stream = io.BytesIO(r.read())
    return img_stream

def get_image_emb(image_url):
    with torch.no_grad():
        image = pimage.open(download_image(image_url))
        image_emb = model.encode_image(preprocess(image).unsqueeze(0).to("cpu"))
        image_emb /= image_emb.norm(dim=-1, keepdim=True)
        image_emb = image_emb.cpu().detach().numpy().astype("float32")[0]
        return image_emb

def get_text_emb(text):
    with torch.no_grad():
        text_emb = model.encode_text(clip.tokenize([text], truncate=True).to("cpu"))
        text_emb /= text_emb.norm(dim=-1, keepdim=True)
        text_emb = text_emb.cpu().detach().numpy().astype("float32")[0]
    return text_emb


(768,)


In [128]:
pos_emb = get_image_emb("https://i.imgur.com/75aBao3.jpg")
neg_emb = get_image_emb("https://i.imgur.com/DdMvFad.jpg")
# print(tp_emb.shape)

vals = clf.predict([pos_emb, neg_emb])
print(vals)

[1. 0.]


In [132]:
pos_x_val = torch.from_numpy(np.load(r"F:\nsfw_testset\drawings-test\img_emb\img_emb_0.npy"))
neg_x_val = torch.from_numpy(np.load(r"F:\nsfw_testset\neutral-test\img_emb\img_emb_0.npy"))

x, y = npy_to_tensor_dataset(pos_x_val, neg_x_val)
val_dataset = TensorDataset(x, y)

# Define the data loaders
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=True)
model.eval()
with torch.cuda.amp.autocast():
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Forward pass
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            outputs_list = outputs.tolist()

            # Compute the loss
            loss = criterion(outputs, labels)


RuntimeError: forward() is missing value for argument 'input'. Declaration: forward(__torch__.multimodal.model.multimodal_transformer.Multimodal self, Tensor image, Tensor input) -> ((Tensor, Tensor))